In [1]:
import pandas as pd
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor
from Bio import SeqIO, Entrez
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import os
import boto3
import time
import re
import json
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.colors import to_hex
from matplotlib.lines import Line2D
import seaborn as sns
STAGING_BUCKET = os.environ['STAGING_BUCKET']
ATHENA_CURSOR = connect(s3_staging_dir=STAGING_BUCKET,
                 region_name="us-west-2",
                 cursor_class=PandasCursor).cursor()
Entrez.email = "jtrimble@agbiome.com"
from bs4 import BeautifulSoup
import lxml

- (Alphaproteobacteria) Azospirillum brasilense - nifH sequence = accession number NZ_VISK01000015.1, gene number 56451760
- (Firmicutes) Paenibacillus polymyxa - nifH sequence = accession number NC_023037.2, gene number 17918479
- (Gammaproteobacteria) Pseudomonas stutzeri - nifH sequence = accession number NC_009434.1, gene number = 5097413
- (Betaproteobacteria) Azoarcus olearius - nifH sequence = accession number NC_008702.1, gene number = 4608650

In [265]:
goi = {
    56451760: 'NZ_VISK01000015.1',
    17918479: 'NC_023037.2',
    5097413: 'NC_009434.1',
    4608650: 'NC_008702.1'
}

In [266]:
wg_ids = []

for wg in goi.values():
    fasta_id = Entrez.read(
        Entrez.esearch(
            db='nucleotide', 
            term=wg, 
            retmax=500, 
            rettype='text'
        )
    )
    wg_ids.append(fasta_id['IdList'][0])
print(wg_ids)

['1719308528', '734699963', '146280397', '119896292']


In [374]:
def retrieve_seq_locations(gene):
    
    """Returns an xml doc with the tags 
    specifying the location of the sequence
    of interest inside a full genome of another
    fasta record
    """

    
    get_data = (
        Entrez.efetch(
            db='gene', 
            id=gene, 
            rettype='gb', 
            retmode='xml'
        )
        .read()
        .decode()
    )
    
    return(get_data)
        
    

In [375]:
def get_fasta(genes_of_interest):
    
    """Builds and uploads a FASTA file of all
    requested sequences by retrieving there location
    from NCBI"""
    
    gene_locs = {}
    
    for gene in genes_of_interest.keys():
        gd = retrieve_seq_locations(gene)
        soup = BeautifulSoup(gd, 'xml')
        fruum = int(soup.find_all('Seq-interval_from')[0].get_text()) + 1
        tooo = int(soup.find_all('Seq-interval_to')[0].get_text()) + 1
        gene_locs[genes_of_interest[gene]] = {gene: [fruum, tooo]}
    
    access = {}
    for accession in gene_locs.keys():
        get_record = Entrez.read(
            Entrez.esearch(
                db='nucleotide', 
                term=accession, 
                retmax=500, 
                rettype='text'
            )
        )
        access[accession] = get_record['IdList'][0]
    
    fastas = []
    for topkey, topvalue in gene_locs.items():
        for lowkey, lowval in topvalue.items():
            gbfile_handle = (
                Entrez.efetch(
                    db="nuccore",
                    id=access[topkey],
                    seq_start=min(lowval), 
                    seq_stop=max(lowval), 
                    rettype='fasta', 
                    parsed=True
                    )
                )
            fastas.append(gbfile_handle.read())
    
    print("Uploading to s3...")
    bf = bytes(''.join(fastas).encode('utf-8'))
    client = boto3.client('s3')
    client.put_object(
        Body=bf, 
        Bucket='agbiome-temp-projects', 
        Key=f'multi-blast-nifH.fasta'
    )
    return(print("Finished..."))

In [341]:
f = get_fasta(goi)

Uploading to s3...
Finished...


In [342]:
def run_batch_job(jtype, jobname, filename):
    batch = boto3.client("batch")
    resp = batch.submit_job(
            jobName=f'{jobname}',
            jobQueue='on-demand-queue',
            jobDefinition="awsbatch_run_blast",
            containerOverrides={
                'vcpus': 64,
                'memory': 450000,
                'environment': [
                    {
                        'name': 'BLASTDB_NAME',
                        'value': 'agbiome-genomes'
                    },
                    {
                        'name': 'BLASTCMD',
                        'value': f'{jtype}'
                    },
                    {
                        'name': 'FILENAME',
                        'value': f'{filename}'
                    },
                    {
                        'name': 'INPUT_BUCKET_S3_URL',
                        'value': 's3://agbiome-temp-projects'
                    },
                    {
                        'name': 'OUTPUT_BUCKET',
                        'value': 's3://agbiome-temp-projects/nifh-blast'
                    },
                    {
                        'name': 'EXTRA_BLAST_CMD_PARAMS',
                        'value': '-evalue 1e-5 -max_target_seqs 1000000'
                    },
                    {
                        'name': 'CPU',
                        'value': '64'
                    }
                ],
            }
        )

    return(resp)

In [343]:
run_batch_job(jtype='blastn', jobname='nifh-blast', filename='multi-blast-nifH.fasta')

{'ResponseMetadata': {'RequestId': '33b40939-21bc-41b6-ae9f-f81613fed4e7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 12 Sep 2022 20:24:25 GMT',
   'content-type': 'application/json',
   'content-length': '160',
   'connection': 'keep-alive',
   'x-amzn-requestid': '33b40939-21bc-41b6-ae9f-f81613fed4e7',
   'access-control-allow-origin': '*',
   'x-amz-apigw-id': 'YXRLEHkYvHcFUzg=',
   'access-control-expose-headers': 'X-amzn-errortype,X-amzn-requestid,X-amzn-errormessage,X-amzn-trace-id,X-amz-apigw-id,date',
   'x-amzn-trace-id': 'Root=1-631f9579-5555014f12ebf9f553357265'},
  'RetryAttempts': 0},
 'jobArn': 'arn:aws:batch:us-west-2:728348960442:job/7f7ef438-d26f-4956-b78d-c309eae95722',
 'jobName': 'nifh-blast',
 'jobId': '7f7ef438-d26f-4956-b78d-c309eae95722'}

In [344]:
! aws s3 cp s3://agbiome-temp-projects/nifh-blast/multi-blast-nifH.agbiome-genomes.blastout.json.gz ../data/

download: s3://agbiome-temp-projects/nifh-blast/multi-blast-nifH.agbiome-genomes.blastout.json.gz to ../data/multi-blast-nifH.agbiome-genomes.blastout.json.gz


In [345]:
df = pd.read_json('../data/multi-blast-nifH.agbiome-genomes.blastout.json.gz', lines=True)

In [347]:
df.groupby('qseqid')['sseqid'].nunique()

qseqid
NC_008702.1:571215-572108          114
NC_009434.1:1432264-1433145        255
NC_023037.2:1087620-1088486         45
NZ_VISK01000015.1:262164-263045    187
Name: sseqid, dtype: int64

In [362]:
def extract_aim(identifier):
    return re.search('AIM\d+', identifier).group()

In [363]:
df['aim'] = df['sseqid'].apply(extract_aim)

In [351]:
df.aim.nunique()

428

In [402]:
aims = tuple(df.aim.unique())

In [417]:
query = f'''
    SELECT env_description_level_3, env_description_level_4, aim_number, afs_number,
    latitude, longitude, origin_country, ncbi_taxonomy_id, taxonomic_classification, completeness, contamination
    from microbial_metadata.aim_browser_backend
    WHERE aim_number in {aims}
'''
aois = ATHENA_CURSOR.execute(query).as_pandas()

In [404]:
aoi2 = aois.loc[aois['ncbi_taxonomy_id'] == 381124].aim_number.unique()

In [405]:
df.loc[df['aim'].isin(aoi2)]

,qseqid,sseqid,stitle,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qseq,sseq,db,tag,db_info,aim
30,NZ_VISK01000015.1:262164-263045,AIM008522_proj1_run7_20140220_plate48_C12_seq2...,AIM008522_proj1_run7_20140220_plate48_C12_seq2...,87.428,867,103,2,10,873,22154,23017,0.000000e+00,992,CGCCAGATTGCGTTCTACGGTAAGGGCGGTATCGGCAAGTCCACCA...,CGCCAGATTGCTTTCTACGGTAAGGGCGGCATCGGCAAGTCCACCA...,agbiome-genomes,8b8e49b49112,NaN,AIM008522
57,NZ_VISK01000015.1:262164-263045,AIM014771_proj1_run26_20150430_plate192_B9_seq...,AIM014771_proj1_run26_20150430_plate192_B9_seq...,85.998,807,113,0,22,828,12282,11476,0.000000e+00,865,TTCTACGGTAAGGGCGGTATCGGCAAGTCCACCACCTCCCAGAACA...,TTCTACGGCAAGGGTGGGATCGGCAAGTCCACCACCTCGCAGAACA...,agbiome-genomes,8b8e49b49112,NaN,AIM014771
68,NZ_VISK01000015.1:262164-263045,AIM014773_proj1_run26_20150430_plate192_D9_seq...,AIM014773_proj1_run26_20150430_plate192_D9_seq...,85.466,805,117,0,22,826,43054,42250,0.000000e+00,839,TTCTACGGTAAGGGCGGTATCGGCAAGTCCACCACCTCCCAGAACA...,TTCTATGGCAAAGGCGGCATCGGCAAATCGACCACGTCGCAGAACA...,agbiome-genomes,8b8e49b49112,NaN,AIM014773
69,NZ_VISK01000015.1:262164-263045,AIM014719_proj1_run26_20150430_plate192_F2_seq...,AIM014719_proj1_run26_20150430_plate192_F2_seq...,85.190,817,121,0,10,826,146316,147132,0.000000e+00,839,CGCCAGATTGCGTTCTACGGTAAGGGCGGTATCGGCAAGTCCACCA...,CGGCAAATCGCCTTCTATGGCAAAGGCGGCATCGGCAAATCGACCA...,agbiome-genomes,8b8e49b49112,NaN,AIM014719
70,NZ_VISK01000015.1:262164-263045,AIM014716_proj1_run26_20150430_plate192_C2_seq...,AIM014716_proj1_run26_20150430_plate192_C2_seq...,85.466,805,117,0,22,826,42101,41297,0.000000e+00,839,TTCTACGGTAAGGGCGGTATCGGCAAGTCCACCACCTCCCAGAACA...,TTCTATGGCAAAGGCGGCATCGGCAAATCGACCACGTCGCAGAACA...,agbiome-genomes,8b8e49b49112,NaN,AIM014716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,NC_008702.1:571215-572108,AIM010390_proj1_run9_20140404_plate69_E1_seq11...,AIM010390_proj1_run9_20140404_plate69_E1_seq11...,79.712,626,112,11,109,728,81203,80587,3.980000e-118,438,ATCGTCGGTTGCGACCCGAAGGCTGACTCCACCCGTCTGATCCTCC...,ATCGTCGGCTGCGATCCCAAGGCTGATTCGACCCGCTTGATCCTGC...,agbiome-genomes,8b8e49b49112,NaN,AIM010390
589,NC_008702.1:571215-572108,AIM057096_proj1_run105_20181016_plate597_G4_se...,AIM057096_proj1_run105_20181016_plate597_G4_se...,75.687,728,162,15,106,827,33823,34541,1.920000e-91,350,ATGATCGTCGGTTGCGACCCGAAGGCTGACTCCACCCGTCTGATCC...,ATGATCGTCGGCTGTGACCCGAAGGCGGACTCCACGCGGCTTATCC...,agbiome-genomes,8b8e49b49112,NaN,AIM057096
594,NC_008702.1:571215-572108,AIM059996_proj1_run106_20181023_plate627_A11_s...,AIM059996_proj1_run106_20181023_plate627_A11_s...,74.730,740,172,14,106,839,20017,19287,1.950000e-81,316,ATGATCGTCGGTTGCGACCCGAAGGCTGACTCCACCCGTCTGATCC...,ATGATCGTCGGCTGCGACCCCAAAGCCGACTCCACCCGCCTTATCT...,agbiome-genomes,8b8e49b49112,NaN,AIM059996
598,NC_008702.1:571215-572108,AIM063976_proj1_run117_20190322_plate669_E9_se...,AIM063976_proj1_run117_20190322_plate669_E9_se...,80.612,392,67,6,106,494,59,444,9.110000e-75,294,ATGATCGTCGGTTGCGACCCGAAGGCTGACTCCACCCGTCTGATCC...,ATGATCGTAGGCTGTGACCCGAAGGCTGACTCCACCCGTCTCATTC...,agbiome-genomes,8b8e49b49112,NaN,AIM063976


In [406]:
maize = aois.loc[aois['ncbi_taxonomy_id'] == 381124]

In [408]:
maize['aim'] = maize['aim_number'].str.strip('AIM').astype(int)

/tmp/ipykernel_2689/1719359557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maize['aim'] = maize['aim_number'].str.strip('AIM').astype(int)


In [421]:
query = f'''
    WITH q1 AS (
        SELECT ROW_NUMBER() 
                OVER(PARTITION BY aim, asm
                ORDER BY asm,
                bitscore DESC) AS row_number,
                aim,
                asm,
                pident,
                bitscore,
                length,
                ssciname AS _16S_id
        FROM "genomics"."blasttargetedloci_parquet"
        WHERE aim in {tuple(maize.aim.unique())}
        ),
    meta AS (
        SELECT CAST(SUBSTRING(aim_number, 4) AS integer) AS aim,
                aim_number,
                afs_number
        FROM "microbial_metadata"."aim_isolates"
        )
    SELECT q.*, m.aim_number
    FROM q1 q
    JOIN meta m ON m.aim = q.aim
    WHERE row_number <= 5
    ORDER BY aim, asm, bitscore DESC
'''
blast = ATHENA_CURSOR.execute(query).as_pandas()

In [425]:
blast.head(10)

,row_number,aim,asm,pident,bitscore,length,_16S_id,aim_number
0,1,6475,5663,99.345,2765,1527,Leclercia adecarboxylata,AIM006475
1,2,6475,5663,99.603,2761,1513,Enterobacter ludwigii,AIM006475
2,3,6475,5663,98.896,2750,1540,Klebsiella aerogenes KCTC 2190,AIM006475
3,4,6475,5663,98.639,2730,1543,Enterobacter cloacae,AIM006475
4,5,6475,5663,98.691,2713,1528,Enterobacter cloacae,AIM006475
5,1,6482,5670,99.345,2765,1527,Leclercia adecarboxylata,AIM006482
6,2,6482,5670,99.603,2761,1513,Enterobacter ludwigii,AIM006482
7,3,6482,5670,98.896,2750,1540,Klebsiella aerogenes KCTC 2190,AIM006482
8,4,6482,5670,98.639,2730,1543,Enterobacter cloacae,AIM006482
9,5,6482,5670,98.691,2713,1528,Enterobacter cloacae,AIM006482


In [433]:
aois.loc[aois['ncbi_taxonomy_id'] == 381124].aim_number.nunique()

45

In [434]:
merged = (
    aois.loc[aois['ncbi_taxonomy_id'] == 381124]
    .merge(blast, on=['aim_number'], how='outer')
    .to_csv('../data/nifH-metadata.csv', index=False)
)

In [429]:
merged

,env_description_level_3,env_description_level_4,aim_number,afs_number,latitude,longitude,origin_country,ncbi_taxonomy_id,taxonomic_classification,completeness,contamination,row_number,aim,asm,pident,bitscore,length,_16S_id
0,plant_rhizosphere,Nematode field (field B8A-B) corn core,AIM037119,AFS082653,35.669711,-78.491692,US,381124.0,Paenibacillus kribbensis,[94.36],[5.17],1,37119,43205,99.100,2595,1444,Paenibacillus peoriae KCTC 3763
1,plant_rhizosphere,Nematode field (field B8A-B) corn core,AIM037119,AFS082653,35.669711,-78.491692,US,381124.0,Paenibacillus kribbensis,[94.36],[5.17],2,37119,43205,99.100,2593,1444,Paenibacillus polymyxa
2,plant_rhizosphere,Nematode field (field B8A-B) corn core,AIM037119,AFS082653,35.669711,-78.491692,US,381124.0,Paenibacillus kribbensis,[94.36],[5.17],3,37119,43205,99.032,2591,1446,Paenibacillus polymyxa
3,plant_rhizosphere,Nematode field (field B8A-B) corn core,AIM037119,AFS082653,35.669711,-78.491692,US,381124.0,Paenibacillus kribbensis,[94.36],[5.17],4,37119,43205,98.893,2580,1446,Paenibacillus polymyxa
4,plant_rhizosphere,Nematode field (field B8A-B) corn core,AIM037119,AFS082653,35.669711,-78.491692,US,381124.0,Paenibacillus kribbensis,[94.36],[5.17],5,37119,43205,98.892,2577,1444,Paenibacillus polymyxa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,plant_corpus,"plant grown in Metro-Mix 360, nonsterile, in A...",AIM014771,AFS056010,35.896928,-78.880882,US,381124.0,NaN,[100.0],[0.0],1,14771,26494,99.468,2734,1504,Ideonella dechloratans
221,plant_corpus,"plant grown in Metro-Mix 360, nonsterile, in A...",AIM014771,AFS056010,35.896928,-78.880882,US,381124.0,NaN,[100.0],[0.0],2,14771,26494,98.125,2603,1493,Ideonella sakaiensis
222,plant_corpus,"plant grown in Metro-Mix 360, nonsterile, in A...",AIM014771,AFS056010,35.896928,-78.880882,US,381124.0,NaN,[100.0],[0.0],3,14771,26494,97.244,2577,1524,Leptothrix mobilis
223,plant_corpus,"plant grown in Metro-Mix 360, nonsterile, in A...",AIM014771,AFS056010,35.896928,-78.880882,US,381124.0,NaN,[100.0],[0.0],4,14771,26494,97.324,2532,1495,Rubrivivax benzoatilyticus JA2 = ATCC BAA-35


In [388]:
aois.loc[aois['ncbi_taxonomy_id'] == 381124].to_csv('../data/nifH-metadata.csv', index=False)

In [385]:
(
    aois.loc[aois['ncbi_taxonomy_id'] == 381124]
    .groupby(['env_description_level_3'])
    ['taxonomic_classification']
    .unique()
    .reset_index()
)

,env_description_level_3,taxonomic_classification
0,plant_corpus,"[Burkholderia vietnamiensis, nan]"
1,plant_rhizosphere,"[nan, Herbaspirillum seropedicae, Bacillus_C m..."


In [386]:
(
    aois.loc[aois['ncbi_taxonomy_id'] == 381124]
    .groupby(['env_description_level_3'])
    ['taxonomic_classification']
    .nunique()
    .reset_index()
)

,env_description_level_3,taxonomic_classification
0,plant_corpus,1
1,plant_rhizosphere,10


In [367]:
df.loc[df['aim'].isin(aoi2)].qseqid.nunique()

4

In [369]:
df.loc[df['aim'].isin(aoi2)].aim.nunique()

45

In [373]:
df.loc[df['aim'].isin(aoi2)].groupby('aim').pident.unique()

aim
AIM006475    [76.048, 79.472]
AIM006482    [76.048, 79.472]
AIM006522    [76.048, 79.472]
AIM006642    [76.048, 79.472]
AIM006699    [84.586, 79.718]
AIM008522            [87.428]
AIM010390    [84.324, 79.712]
AIM014716            [85.466]
AIM014718            [85.217]
AIM014719             [85.19]
AIM014737            [85.217]
AIM014771    [85.998, 78.171]
AIM014773            [85.466]
AIM014775    [83.413, 78.082]
AIM014788            [84.302]
AIM018069    [84.146, 77.497]
AIM018099    [84.211, 77.497]
AIM018100    [84.146, 77.497]
AIM023501            [99.193]
AIM031715            [82.569]
AIM031716            [83.431]
AIM031722            [82.569]
AIM031857            [84.375]
AIM031887            [76.178]
AIM037119             [95.04]
AIM037221    [84.685, 79.196]
AIM037222    [84.211, 79.408]
AIM037353     [84.23, 79.635]
AIM037389    [84.685, 79.196]
AIM037424    [84.211, 79.408]
AIM037715            [95.354]
AIM037716             [95.04]
AIM057096            [75.687]
AIM059